# Analysis of lblnew-bestfit results  by (molecule, band)

In [122]:
%reload_ext autoreload
%autoreload 2

import os
import copy
import pprint
import pandas as pd
import pymongo

from climatools.atm.absorbers import *
from climatools.cliradlw.utils import *
from climatools.lblnew.bestfit_params import *
from climatools.lblnew.pipeline import *
import climatools.lblnew.setup_bestfit as setup_bestfit

from IPython import display

In [123]:
pd.set_option('max_colwidth', -1)

In [124]:
ATMPROS = [('mls', 294), ('saw', 257), ('trp', 300)]

In [125]:
client = pymongo.MongoClient('localhost', 27017)

In [126]:
def make_query(param=None):
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
        else:
            q[f'param.{n}'] = v
    return q

def run_exists(param=None):
    q = make_query(param)
    cnt = client.lblnew.bestfit_lw.count(q)
    return True if cnt else False

def notebook_exists(param=None):
    d = get_analysis_dir(param=param, setup=setup_bestfit)
    try: 
        return 'results.ipynb' in os.listdir(d)
    except FileNotFoundError:
        return False

In [127]:
def get_3tuple_params():
    bandmols = [(band, mol) for band, molconc in nongreys_byband().items()
                for mol in molconc.keys()]
    bandmols = [(mapband_new2old()[band], mol) for band, mol in bandmols]
    
    params = []
    for band, mol in bandmols:
        p = kdist_params(molecule=mol, band=band)

        for k in ('option_compute_ktable', 'option_compute_btable', 'commitnumber'):
            if k in p: del p[k]
        
        param_bandmol = []
        for atmpro, tsfc in ATMPROS:
            p['atmpro'], p['tsfc'] = atmpro, tsfc
            
            q = make_query(p)
            r = client.lblnew.bestfit_lw.find_one(q, projection=['param'])
            if not r:
                print('Following lblnew run not available:')
                pprint.pprint(p)
                continue
                
            if not notebook_exists(param=r['param']):
                print('Following lblnew analysis not avaialble:')
                pprint.pprint(p)
                continue
                
            param_bandmol.append(r.get('param'))
        params.append(tuple(param_bandmol))
    return params

In [128]:
def get_hrefs(param_3tuple=None):
    hrefs = []
    for param in param_3tuple:
        if notebook_exists(param=param):
            url = nbviewer_url(param=param, setup=setup_bestfit)
            s = f'''<a href="{url}">{param['atmpro']}</a>'''
            hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs

def tb_hrefs():
    df = pd.DataFrame()
    for params in get_3tuple_params():
        hrefs = get_hrefs(param_3tuple=params)
        molecule, band = params[0]['molecule'], params[0]['band']
        df.loc[molecule, band] = hrefs
        
    df.fillna('', inplace=True)
    clirad_bands = [mapband_old2new()[b] for b in df.columns]
    columns = pd.MultiIndex.from_arrays([df.columns, clirad_bands],
                                        names=['lblnew', 'clirad'])
    df.columns = columns
    return df

In [129]:
def script():
    df = tb_hrefs()
    s_html = df.to_html(escape=False)
    display.display(display.HTML(s_html))

In [130]:
script()

lblnew,1,2,3a,3b,3c,4,5,6,7,8,9
clirad,1,2,3,4,5,6,7,8,9,10,11
h2o,"<a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band01_wn_20_340/nv_1000/dv_0.001/ng_12/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__3__6/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.2_0.2_0.2__0.2_0.2_0.2__0.2_0.2_0.65_0.65_0.65_0.65/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66_1.66__1.66_1.66_1.5_1.5_1.5_1.5/option_compute_ktable_0/option_compute_btable_0/crd_5014a19/results.ipynb"">mls <a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band01_wn_20_340/nv_1000/dv_0.001/ng_12/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__3__6/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_saw/wgt_k_1/wgt_0.2_0.2_0.2__0.2_0.2_0.2__0.2_0.2_0.65_0.65_0.65_0.65/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66_1.66__1.66_1.66_1.5_1.5_1.5_1.5/option_compute_ktable_0/option_compute_btable_0/crd_5014a19/results.ipynb"">saw <a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band01_wn_20_340/nv_1000/dv_0.001/ng_12/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__3__6/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_trp/wgt_k_1/wgt_0.2_0.2_0.2__0.2_0.2_0.2__0.2_0.2_0.65_0.65_0.65_0.65/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66_1.66__1.66_1.66_1.5_1.5_1.5_1.5/option_compute_ktable_0/option_compute_btable_0/crd_5014a19/results.ipynb"">trp","<a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band02_wn_340_540/nv_1000/dv_0.001/ng_11/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_2__3__6/ng_adju_-2__-2__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.6_0.6__0.6_0.6_0.6__0.6_0.6_0.6_0.6_0.6_0.6/wgt_flux_1/w_diffuse_1.66_1.66__1.8_1.8_1.8__1.8_1.66_1.45_1.45_1.45_1.45/option_compute_ktable_0/option_compute_btable_0/crd_5014a19/results.ipynb"">mls <a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band02_wn_340_540/nv_1000/dv_0.001/ng_11/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_2__3__6/ng_adju_-2__-2__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_saw/wgt_k_1/wgt_0.6_0.6__0.6_0.6_0.6__0.6_0.6_0.6_0.6_0.6_0.6/wgt_flux_1/w_diffuse_1.66_1.66__1.8_1.8_1.8__1.8_1.66_1.45_1.45_1.45_1.45/option_compute_ktable_0/option_compute_btable_0/crd_5014a19/results.ipynb"">saw <a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band02_wn_340_540/nv_1000/dv_0.001/ng_11/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_2__3__6/ng_adju_-2__-2__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_trp/wgt_k_1/wgt_0.6_0.6__0.6_0.6_0.6__0.6_0.6_0.6_0.6_0.6_0.6/wgt_flux_1/w_diffuse_1.66_1.66__1.8_1.8_1.8__1.8_1.66_1.45_1.45_1.45_1.45/option_compute_ktable_0/option_compute_btable_0/crd_5014a19/results.ipynb"">trp",mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,"<a href=""http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/h2o/conc_None/band08_wn_1380_1900/nv_1000/dv_0.001/ng_8/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250__P_500_T_250/ng_refs_3__2__3/ng_adju_0__-1__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.55_0.55_0.85__0.85_0.85__0_0.3_0.55/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66__1.66_1.6

In [131]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')